# Task 3 - AES
Felix Kleinsteuber, 185 709

AES ist eine rundenbasierte Blockchiffre und besteht aus folgenden Subfunktionen, die hier implementiert und zu AES Encrypt/Decrypt kombiniert werden sollen:

1. Schlüsseladdition (XOR)
2. SubBytes (Lokale nichtlineare Substitution)
3. ShiftRows (Globale Permutation)
4. MixColumns (Globale lineare Substitution)

Ein **Block** wird im folgenden als 4x4-Byte-Matrix interpretiert (d.h. pro Block 16 Byte = 128 bit). Wir benötigen daher eine Konvertierungsfunktion zwischen bytes/strings und einer solchen Matrix.

## 1. S-Boxes importieren
Die Dateien sind in einem CSV-ähnlichen Format gegeben. Die Einträge sind als Hexadezimalzahlen angegeben und müssen in Bytes konvertiert werden.

In [1]:
import re
import numpy as np

def read_sbox_file(filename):
    with open(filename, "r") as f:
        content = f.read()
        # Werte in einer Liste der Länge 256 voneinander isolieren
        vals = re.split(",\s", content)
        # Werte als Hexadezimalzahlen parsen
        int_vals = [int(val, 16) for val in vals]
        # Zu numpy Array konvertieren (ein Wert = 1 byte = np.uint8) und reshape
        return np.array(int_vals, dtype=np.uint8).reshape((16, 16))

sbox = read_sbox_file("SBox.txt")
assert sbox[0,0] == 0x63
sbox_inv = read_sbox_file("SBoxInvers.txt")
assert sbox_inv[0,0] == 0x52

## 2. Initialisierung
Wir möchten einen Eingabestring der Länge 16 in eine 4x4 Matrix mit je einem Byte konvertieren (d.h. ein char wird in ein Byte konvertiert). Die Anordnung der Bytes erfolgt wie in den Folien zuerst auf die Spalten, dann auf die Zeilen.

In [2]:
def str_to_matrix(input_str: str):
    assert len(input_str) == 16
    # Jedes Zeichen wird in ein byte konvertiert, der entstehende Array in eine 4x4-Matrix umgeformt (transpose, da spaltenweise!)
    # Es dürfen nur ASCII-Zeichen vorkommen! (ord < 256)
    return np.array([ord(c) for c in input_str], dtype=np.uint8).reshape((4,4)).transpose()

def matrix_to_str(mat):
    assert mat.shape == (4, 4)
    # Lies alle Zeichen spaltenweise aus und konkateniere sie
    chars = [chr(c) for c in np.nditer(mat.transpose(), order="C")]
    return "".join(chars)

# Test
input_str = "Das ist ein Test"#"Hello Crypto! :)"
mat = str_to_matrix(input_str)
print(mat)
out_str = matrix_to_str(mat)
print(out_str)
assert input_str == out_str

[[ 68 105 101  84]
 [ 97 115 105 101]
 [115 116 110 115]
 [ 32  32  32 116]]
Das ist ein Test


## 3. Rundenschlüssel addieren (X-OR)

In [3]:
def add_round_key(mat, key):
    return np.bitwise_xor(mat, key)

# Test
test_mat = np.array(mat, dtype=np.uint8)
test_key = str_to_matrix("1234567890123456")
test_mat_key = add_round_key(test_mat, test_key)
print(test_mat_key)
test_mat_key_inv = add_round_key(test_mat_key, test_key)
print(test_mat_key_inv)
assert np.all(test_mat == test_mat_key_inv)

[[117  92  92 103]
 [ 83  69  89  81]
 [ 64  67  95  70]
 [ 20  24  18  66]]
[[ 68 105 101  84]
 [ 97 115 105 101]
 [115 116 110 115]
 [ 32  32  32 116]]


## 4. SubBytes (Lokale Substitution mit S-Boxen)
Die Herkunft der S-Box bei AES ist unklar, d.h. sie wird als gegebene feste Tabelle behandelt. Substituiert wird, indem für jedes Byte der entsprechende Substitutionswert aus der Tabelle (s_box bzw. sbox_inv Array) gelesen wird.

In [4]:
# Zugriff auf S-Box für einzelnes Byte
def get_sbox(val, inv):
    # Zeile ergibt sich aus den ersten 4 bits
    s_row = (val & 0xf0) >> 4
    # Spalte ergibt sich aus den hinteren 4 bits
    s_col = val & 0xf
    return sbox_inv[s_row, s_col] if inv else sbox[s_row, s_col]

def sub_bytes(mat, inv=False):
    # Substituiert alle Elemente in der Matrix durch S-Box-Wert
    # Durch Setzen von inv=True wird die inverse S-Box genutzt
    for row in range(4):
        for col in range(4):
            mat[row, col] = get_sbox(mat[row, col], inv)
    return mat

# Test
test_mat = np.array(mat, dtype=np.uint8)
sub_mat = sub_bytes(test_mat)
print(sub_mat)
subinv_mat = sub_bytes(sub_mat, inv=True)
print(subinv_mat)
assert np.all(test_mat == subinv_mat)

[[ 27 249  77  32]
 [239 143 249  77]
 [143 146 159 143]
 [183 183 183 146]]
[[ 68 105 101  84]
 [ 97 115 105 101]
 [115 116 110 115]
 [ 32  32  32 116]]


## 5. ShiftRows
Zeilenweise Permutation. Verschlüsselung: Zyklische Verschiebung nach links, Entschlüsselung: nach rechts.

In [5]:
def shift_rows(mat, inv=False):
    if inv:
        # Zyklische Verschiebung nach rechts
        mat[1,:] = mat[1,[3,0,1,2]]
        mat[2,:] = mat[2,[2,3,0,1]]
        mat[3,:] = mat[3,[1,2,3,0]]
    else:
        # Zyklische Verschiebung nach links
        mat[1,:] = mat[1,[1,2,3,0]]
        mat[2,:] = mat[2,[2,3,0,1]]
        mat[3,:] = mat[3,[3,0,1,2]]
    return mat

# Test
test_mat = np.array(mat, dtype=np.uint8)
shifted_mat = shift_rows(test_mat)
print(shifted_mat)
shiftedinv_mat = shift_rows(shifted_mat, inv=True)
print(shiftedinv_mat)
assert np.all(test_mat == shiftedinv_mat)

[[ 68 105 101  84]
 [115 105 101  97]
 [110 115 115 116]
 [116  32  32  32]]
[[ 68 105 101  84]
 [ 97 115 105 101]
 [115 116 110 115]
 [ 32  32  32 116]]


## 6. MixColumns
MixColumns entspricht einer Matrixmultiplikation im Galois-Körper. Zur Multiplikation mit $p(x) = x$ wird die Funktion xtimes genutzt, die Addition von Polynomen entspricht bitweisem XOR. Durch diese beiden Operationen können beliebige Polynome miteinander multipliziert werden.

In [6]:
# gemaess Foliendefinition, multipliziert Polynom a(x) mit Polynom p(x) = x
def xtimes(a):
    t = a << 1
    if a & 0b10000000 != 0:
        t = np.bitwise_xor(t, 0x1b)
    return t

# Test
test_a = 0b00101010
print(bin(xtimes(test_a)))
test_a = 0b10101100
print(bin(xtimes(test_a)))

0b1010100
0b101000011


In [7]:
# Matrizen zur Multiplikation
# Jeder Eintrag codiert ein Polynom (1 = 1, 2 = x, 3 = x + 1)
mc_mat = np.array([
    [2, 3, 1, 1],
    [1, 2, 3, 1],
    [1, 1, 2, 3],
    [3, 1, 1, 2]], dtype=np.uint8)
# Inverses von mc_mat im Galois-Körper
mc_mat_inv = np.array([
    [0xe, 0xb, 0xd, 0x9],
    [0x9, 0xe, 0xb, 0xd],
    [0xd, 0x9, 0xe, 0xb],
    [0xb, 0xd, 0x9, 0xe]], dtype=np.uint8)

In [8]:
# Funktion zum Ausfuehren der Matrixmultiplikation auf einer einzelnen Spalte
def mix_single_col(in_col, inv):
    global mc_mat, mc_mat_inv
    mat = mc_mat_inv if inv else mc_mat
    out_col = np.zeros(in_col.shape, dtype=np.uint8)
    # Iteriere durch Matrix
    for row in range(4):
        for col in range(4):
            # Iteriere durch bits in mat[row, col]
            for bitpos in range(8):
                if mat[row, col] & (1 << bitpos) != 0:
                    # Multipliziere einstelliges Polynom (z.B. x^2 = x * x) mit Eingabepolynom
                    prod = in_col[col]
                    for _ in range(bitpos):
                        prod = xtimes(prod)
                    # Addiere das Produkt zum Ausgabepolynom
                    out_col[row] = np.bitwise_xor(out_col[row], prod)
    return out_col

def mix_columns(mat, inv=False):
    for col in range(4):
        mat[:,col] = mix_single_col(mat[:,col], inv)
    return mat

# Test
test_mat = np.array(mat, dtype=np.uint8)
print(test_mat)
mixed_test_mat = mix_columns(test_mat)
print(mixed_test_mat)
mixed_test_mat_inv = mix_columns(mixed_test_mat, inv=True)
print(mixed_test_mat_inv)
assert np.all(test_mat == mixed_test_mat_inv)

[[ 68 105 101  84]
 [ 97 115 105 101]
 [115 116 110 115]
 [ 32  32  32 116]]
[[120  19  63   0]
 [ 51  51  37 127]
 [163 146 176  75]
 [158 252 232   2]]
[[ 68 105 101  84]
 [ 97 115 105 101]
 [115 116 110 115]
 [ 32  32  32 116]]


## 7. Alles zusammen - AES Verschlüsselungs- und Entschlüsselungsroutinen
Ver- und Entschlüsselung laufen in jeweils 10 Runden. In der letzten Runde wird kein MixColumns ausgeführt. Vor der ersten Runde wird bereits eine Schlüsseladdition ausgeführt, daher benötigen wir 11 Rundenschlüssel.

Die Rundenschlüssel werden in der nächsten Aufgabe generiert.

In [9]:
def aes_encrypt(mat, keys, verbose=False):
    assert mat.shape == (4,4)
    assert keys.shape == (11, 4, 4)
    enc_mat = add_round_key(np.array(mat, dtype=np.uint8), keys[0])
    for i in range(1, 10):
        enc_mat = sub_bytes(enc_mat)
        enc_mat = shift_rows(enc_mat)
        enc_mat = mix_columns(enc_mat)
        enc_mat = add_round_key(enc_mat, keys[i])
        if verbose:
            print(i, matrix_to_str(enc_mat))
    enc_mat = sub_bytes(enc_mat)
    enc_mat = shift_rows(enc_mat)
    enc_mat = add_round_key(enc_mat, keys[10])
    return enc_mat

# Test
test_mat = np.array(mat, dtype=np.uint8)
print(test_mat)
print(matrix_to_str(test_mat))
keys = np.random.randint(0, 255, size=(11, 4, 4), dtype=np.uint8)
enc_test_mat = aes_encrypt(test_mat, keys)
print(enc_test_mat)
print(matrix_to_str(enc_test_mat))

[[ 68 105 101  84]
 [ 97 115 105 101]
 [115 116 110 115]
 [ 32  32  32 116]]
Das ist ein Test
[[119 216  26 252]
 [167 142  42 247]
 [ 99 138 166 234]
 [114 132  87  10]]
w§crØ*¦Wü÷ê



Decrypt führt alle Operationen in umgekehrter Reihenfolge aus.

In [10]:
def aes_decrypt(mat, keys, verbose=False):
    assert mat.shape == (4,4)
    assert keys.shape == (11, 4, 4)
    # xor muss nicht umgekehrt werden (in x-or key = out <=> out x-or key = in)
    dec_mat = add_round_key(np.array(mat, dtype=np.uint8), keys[10])
    dec_mat = shift_rows(dec_mat, inv=True)
    dec_mat = sub_bytes(dec_mat, inv=True)
    for i in range(9, 0, -1):
        if verbose:
            print(i, matrix_to_str(enc_mat))
        dec_mat = add_round_key(dec_mat, keys[i])
        dec_mat = mix_columns(dec_mat, inv=True)
        dec_mat = shift_rows(dec_mat, inv=True)
        dec_mat = sub_bytes(dec_mat, inv=True)
    dec_mat = add_round_key(dec_mat, keys[0])
    return dec_mat

# Test
dec_test_mat = aes_decrypt(enc_test_mat, keys)
print(dec_test_mat)
print(matrix_to_str(dec_test_mat))
assert np.all(dec_test_mat == test_mat)

[[ 68 105 101  84]
 [ 97 115 105 101]
 [115 116 110 115]
 [ 32  32  32 116]]
Das ist ein Test


Klappt!